In [1]:
import time
import os.path
import argparse
import numpy as np
import matplotlib.pyplot as plt
from UNet_model import *
from PIL import Image
import torchvision

In [2]:
# our transform that is applied to all incoming images
transform_image = torchvision.transforms.Compose([
    torchvision.transforms.Resize(size=(128, 256), interpolation=Image.BILINEAR),
    torchvision.transforms.ToTensor()
])

/users/adcy347/.local/lib/python3.9/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


In [3]:
# load the images in the folder (Test Images)
img_data = torchvision.datasets.ImageFolder(root="./predictionImages", transform=transform_image)
img_batch = torch.utils.data.DataLoader(img_data, batch_size=64, shuffle=False, num_workers=4)
print(img_data)

Dataset ImageFolder
    Number of datapoints: 3
    Root location: ./predictionImages
    StandardTransform
Transform: Compose(
               Resize(size=(128, 256), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )


/users/adcy347/.local/lib/python3.9/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [4]:
# load pretrained model if it is there
print("loading unet model...")
file_model = './checkpoints_epochs/train_UNet.pkl'
if os.path.isfile(file_model):
    generator = torch.load(file_model)
    print("    - model restored from file....")
    print("    - filename = %s" % file_model)
else:
    print("unable to load unet.pkl model file")
    exit()


loading unet model...
    - model restored from file....
    - filename = ./checkpoints_epochs/train_UNet.pkl


In [5]:
# make the result directory
if not os.path.exists('./predict/'):
    os.makedirs('./predict/')

In [6]:
# Loop through the dataset and evaluate how well the network predicts
print("\nevaluating network (will take a while)...")
for idx_batch, (imagergb, labelrgb) in enumerate(img_batch):

    # send to the GPU and do a forward pass
    x = Variable(imagergb).cuda(0)
    y = generator.forward(x)

    # max over the classes should be the prediction
    # our prediction is [N, classes, W, H]
    # so we max over the second dimension and take the max response
    pred_class = torch.zeros((y.size()[0], y.size()[2], y.size()[3]))
    for idx in range(0, y.size()[0]):
        pred_class[idx] = torch.argmax(y[idx], dim=0).cpu().int()
    
    # unsqueese so we have [N, 1, W, H] size
    # this allows for debug saving of the images to file...
    pred_class = pred_class.unsqueeze(1).float()
    
    # debug saving generated classes to file
    v_utils.save_image(pred_class.float()/y.size()[1], "./predict/gen_image_{}_{}.png".format(0, idx_batch))
    v_utils.save_image(x.cpu().data, "./predict/original_image_{}_{}.png".format(0, idx_batch))


evaluating network (will take a while)...
